<a href="https://colab.research.google.com/github/wenjunsun/Covid-19-analysis-with-uw-ubicomp/blob/master/2020-11/PSW_weekdays/prepare_google_mobility_data_weekdays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will prepare the dataset for propensity score matching with Google's Mobility dataset, **only extracting the weekday data.**, because we suspect the peaks of behavior on weekends will do something funky to our analysis.

# 0. go to directory containing data + load packages

In [1]:
cd drive/My\ Drive/covid/PSM/data

/content/drive/My Drive/covid/PSM/data


In [2]:
ls

agg_social_dist_2.csv
agg_social_dist.csv
avg_google_mobility_up_to_6_1.csv
county_data_with_reduced_covariates_with_SIP.csv
final_data_google_and_safe_graph_up_to_6_1.csv
google_mobility.csv
OxCGRT_latest.csv
social_dist_aggregated_on_county.csv


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. load + clean data

In [4]:
data = pd.read_csv('google_mobility.csv', parse_dates=['date'],
                   infer_datetime_format = True)

In [5]:
data.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0
2,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-17,6.0,0.0,28.0,-9.0,-24.0,5.0
3,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-18,0.0,-1.0,6.0,1.0,0.0,1.0
4,US,United States,NaN,NaN,NaN,NaN,NaN,2020-02-19,2.0,0.0,8.0,1.0,1.0,0.0


In [6]:
data.dtypes

country_region_code                                           object
country_region                                                object
sub_region_1                                                  object
sub_region_2                                                  object
metro_area                                                   float64
iso_3166_2_code                                               object
census_fips_code                                             float64
date                                                  datetime64[ns]
retail_and_recreation_percent_change_from_baseline           float64
grocery_and_pharmacy_percent_change_from_baseline            float64
parks_percent_change_from_baseline                           float64
transit_stations_percent_change_from_baseline                float64
workplaces_percent_change_from_baseline                      float64
residential_percent_change_from_baseline                     float64
dtype: object

So this dataset is already in a form that has every county's behavioral data in each date from mid February to current date. We want to get the average of behavioral data of each county from Febuary to `6-1-2020`, as we did with SafeGraph analysis.

We can see some data have null `census_fips_code`?

In [7]:
num_counties = len(data['census_fips_code'].unique())
print(f'there are {num_counties} unique counties within Google mobility dataset')

there are 2834 unique counties within Google mobility dataset


In [8]:
data.dropna(subset=['census_fips_code'], inplace = True)

In [9]:
data['census_fips_code'].isnull().sum()

0

In [10]:
# drop unnecessary columns
data.drop(['country_region_code', 'country_region', 'sub_region_1',
           'sub_region_2', 'metro_area', 'iso_3166_2_code'], axis = 1, inplace = True)

In [11]:
data.head()

,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
532,1001.0,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN
533,1001.0,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN
534,1001.0,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0
535,1001.0,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0
536,1001.0,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0


# 2. filter data with non-desirable dates

Now we will delete all the rows which have dates after `6-1-2020`

In [28]:
# default date to filter is 6-1-2020
# given a dataframe and a column that denotes the date column,
# and a date to filter by, return a new dataframe such that the new one
# doesn't contain any rows after the date parameter.
def filterDatesAfter(date = np.datetime64('2020-06-01'), dataset = data, dateColumn = 'date'):

  # helper function that given a row, return true if this row's date is after date. 
  def isThisRowAfterDate(row, date = date):
    return row[dateColumn] > date
  
  newData = dataset[dataset.apply(lambda row: not isThisRowAfterDate(row), axis = 1)]

  return newData

In [29]:
data = filterDatesAfter()

In [30]:
data['date'].unique()

array(['2020-02-15T00:00:00.000000000', '2020-02-16T00:00:00.000000000',
       '2020-02-17T00:00:00.000000000', '2020-02-18T00:00:00.000000000',
       '2020-02-19T00:00:00.000000000', '2020-02-20T00:00:00.000000000',
       '2020-02-21T00:00:00.000000000', '2020-02-22T00:00:00.000000000',
       '2020-02-23T00:00:00.000000000', '2020-02-24T00:00:00.000000000',
       '2020-02-25T00:00:00.000000000', '2020-02-26T00:00:00.000000000',
       '2020-02-27T00:00:00.000000000', '2020-02-28T00:00:00.000000000',
       '2020-02-29T00:00:00.000000000', '2020-03-01T00:00:00.000000000',
       '2020-03-02T00:00:00.000000000', '2020-03-03T00:00:00.000000000',
       '2020-03-04T00:00:00.000000000', '2020-03-05T00:00:00.000000000',
       '2020-03-06T00:00:00.000000000', '2020-03-07T00:00:00.000000000',
       '2020-03-08T00:00:00.000000000', '2020-03-09T00:00:00.000000000',
       '2020-03-10T00:00:00.000000000', '2020-03-11T00:00:00.000000000',
       '2020-03-12T00:00:00.000000000', '2020-03-13

# 3. extract only weekday data.

Now we want to extract only the weekday data. To do this we use `date.weekday()` method.

In [34]:
data

,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
532,1001.0,2020-02-15,5.0,7.0,NaN,NaN,-4.0,NaN
533,1001.0,2020-02-16,0.0,1.0,-23.0,NaN,-4.0,NaN
534,1001.0,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0
535,1001.0,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0
536,1001.0,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0
...,...,...,...,...,...,...,...,...
668361,56045.0,2020-05-26,NaN,NaN,NaN,NaN,-27.0,NaN
668362,56045.0,2020-05-27,NaN,NaN,NaN,NaN,-26.0,NaN
668363,56045.0,2020-05-28,NaN,NaN,NaN,NaN,-22.0,NaN
668364,56045.0,2020-05-29,NaN,NaN,NaN,NaN,-24.0,NaN


In [31]:
date = data.iloc[0]['date']

In [32]:
date

Timestamp('2020-02-15 00:00:00')

In [33]:
# date.weekday() returns int: 0 (Monday), 1 (Tuesday), ..., 6 (Sunday)
date.weekday()

5

In [35]:
def extractWeekdays(dataset = data, dateColumn = 'date'):
  # helper function that given a row, return true if this row's date is a weekday.
  def isThisRowWeekday(row):
    return row[dateColumn].weekday() <= 4
  
  newData = dataset[dataset.apply(lambda row: isThisRowWeekday(row), axis = 1)]

  return newData

In [39]:
data = extractWeekdays(dataset = data, dateColumn= 'date')

In [40]:
data['date'].unique()

array(['2020-02-17T00:00:00.000000000', '2020-02-18T00:00:00.000000000',
       '2020-02-19T00:00:00.000000000', '2020-02-20T00:00:00.000000000',
       '2020-02-21T00:00:00.000000000', '2020-02-24T00:00:00.000000000',
       '2020-02-25T00:00:00.000000000', '2020-02-26T00:00:00.000000000',
       '2020-02-27T00:00:00.000000000', '2020-02-28T00:00:00.000000000',
       '2020-03-02T00:00:00.000000000', '2020-03-03T00:00:00.000000000',
       '2020-03-04T00:00:00.000000000', '2020-03-05T00:00:00.000000000',
       '2020-03-06T00:00:00.000000000', '2020-03-09T00:00:00.000000000',
       '2020-03-10T00:00:00.000000000', '2020-03-11T00:00:00.000000000',
       '2020-03-12T00:00:00.000000000', '2020-03-13T00:00:00.000000000',
       '2020-03-16T00:00:00.000000000', '2020-03-17T00:00:00.000000000',
       '2020-03-18T00:00:00.000000000', '2020-03-19T00:00:00.000000000',
       '2020-03-20T00:00:00.000000000', '2020-03-23T00:00:00.000000000',
       '2020-03-24T00:00:00.000000000', '2020-03-25

As we can see above, we see dates go from `02-17`, `02-18`, `02-19`, `02-20`, `02-21` to `02-24`, which makes sense, because the two dates lost are probably weekends. This validates that our function indeed extracts the weekday data.

# 4. Average behavioral data for each county from Feb to June

In [41]:
data.head()

,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
534,1001.0,2020-02-17,8.0,0.0,NaN,NaN,-27.0,5.0
535,1001.0,2020-02-18,-2.0,0.0,NaN,NaN,2.0,0.0
536,1001.0,2020-02-19,-2.0,0.0,NaN,NaN,2.0,0.0
537,1001.0,2020-02-20,-8.0,-3.0,NaN,NaN,1.0,2.0
538,1001.0,2020-02-21,-3.0,-2.0,NaN,NaN,3.0,0.0


In [42]:
data.drop(['date'], axis = 1, inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [43]:
data.head()

,census_fips_code,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
534,1001.0,8.0,0.0,NaN,NaN,-27.0,5.0
535,1001.0,-2.0,0.0,NaN,NaN,2.0,0.0
536,1001.0,-2.0,0.0,NaN,NaN,2.0,0.0
537,1001.0,-8.0,-3.0,NaN,NaN,1.0,2.0
538,1001.0,-3.0,-2.0,NaN,NaN,3.0,0.0


In [44]:
data = data.groupby(['census_fips_code']).agg(
    avg_retail_and_recreation_percent_change_6_1 = ('retail_and_recreation_percent_change_from_baseline', 'mean'),
    avg_grocery_and_pharmacy_percent_change_6_1 = ('grocery_and_pharmacy_percent_change_from_baseline', 'mean'),
    avg_parks_percent_change_6_1 = ('parks_percent_change_from_baseline', 'mean'),
    avg_transit_stations_percent_change_6_1 = ('transit_stations_percent_change_from_baseline', 'mean'),
    avg_workplaces_percent_change_6_1 = ('workplaces_percent_change_from_baseline', 'mean'),
    avg_residential_percent_change_6_1 = ('residential_percent_change_from_baseline', 'mean')
).reset_index()

In [45]:
data

,census_fips_code,avg_retail_and_recreation_percent_change_6_1,avg_grocery_and_pharmacy_percent_change_6_1,avg_parks_percent_change_6_1,avg_transit_stations_percent_change_6_1,avg_workplaces_percent_change_6_1,avg_residential_percent_change_6_1
0,1001.0,-6.973684,10.026316,NaN,NaN,-25.157895,9.480000
1,1003.0,-13.131579,4.697368,18.578947,-8.197368,-22.618421,7.605263
2,1005.0,3.755102,-7.763158,NaN,NaN,-15.631579,NaN
3,1007.0,1.971429,6.350000,NaN,NaN,-19.907895,NaN
4,1009.0,-5.929825,5.105263,NaN,NaN,-23.565789,8.720000
...,...,...,...,...,...,...,...
2823,56037.0,-10.394737,7.426230,NaN,14.578947,-23.197368,7.120000
2824,56039.0,-38.263158,-17.906977,-24.197368,-41.833333,-35.434211,NaN
2825,56041.0,1.511111,15.942857,NaN,14.842105,-19.421053,NaN
2826,56043.0,-4.400000,-7.000000,NaN,NaN,-20.960526,NaN


In [46]:
# now we are done! save the data!
data.to_csv('avg_google_mobility_up_to_6_1_weekdays.csv', index = False)

# 5. Combine Google Behavioral data with our covariates.

In [47]:
google_data = data

In [48]:
safe_graph_data = pd.read_csv('county_data_with_reduced_covariates_with_SIP.csv')

In [49]:
google_data

,census_fips_code,avg_retail_and_recreation_percent_change_6_1,avg_grocery_and_pharmacy_percent_change_6_1,avg_parks_percent_change_6_1,avg_transit_stations_percent_change_6_1,avg_workplaces_percent_change_6_1,avg_residential_percent_change_6_1
0,1001.0,-6.973684,10.026316,NaN,NaN,-25.157895,9.480000
1,1003.0,-13.131579,4.697368,18.578947,-8.197368,-22.618421,7.605263
2,1005.0,3.755102,-7.763158,NaN,NaN,-15.631579,NaN
3,1007.0,1.971429,6.350000,NaN,NaN,-19.907895,NaN
4,1009.0,-5.929825,5.105263,NaN,NaN,-23.565789,8.720000
...,...,...,...,...,...,...,...
2823,56037.0,-10.394737,7.426230,NaN,14.578947,-23.197368,7.120000
2824,56039.0,-38.263158,-17.906977,-24.197368,-41.833333,-35.434211,NaN
2825,56041.0,1.511111,15.942857,NaN,14.842105,-19.421053,NaN
2826,56043.0,-4.400000,-7.000000,NaN,NaN,-20.960526,NaN


In [50]:
safe_graph_data

,state,state_code,State Name,cnamelong,county_code,diff_in_perc_at_home,SIP?,Median Household Income,% Rural,Population_y,political_diff,% less than 18 years of age,% 65 and over,% Asian,% Black,% Hispanic,% Non-Hispanic White
0,1.0,AL,Alabama,Autauga County,1001.0,0.050678,1,59338.0,42.0,55601,-0.494789,23.7,15.6,1.2,19.3,3.0,74.3
1,1.0,AL,Alabama,Baldwin County,1003.0,0.050312,1,57588.0,42.3,218022,-0.577862,21.6,20.4,1.2,8.8,4.6,83.1
2,1.0,AL,Alabama,Barbour County,1005.0,0.007037,1,34382.0,67.8,24881,-0.056112,20.9,19.4,0.5,48.0,4.3,45.6
3,1.0,AL,Alabama,Bibb County,1007.0,0.011809,1,46064.0,68.4,22400,-0.555441,20.5,16.5,0.2,21.1,2.6,74.6
4,1.0,AL,Alabama,Blount County,1009.0,0.038890,1,50412.0,90.0,57840,-0.813820,23.2,18.2,0.3,1.5,9.6,86.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991,56.0,WY,Wyoming,Sweetwater County,56037.0,-0.018485,0,73315.0,10.9,43051,-0.535382,26.2,12.1,1.0,1.1,16.1,79.3
2992,56.0,WY,Wyoming,Teton County,56039.0,0.075183,1,99087.0,46.4,23081,0.278663,18.4,15.4,1.4,0.6,14.9,81.5
2993,56.0,WY,Wyoming,Uinta County,56041.0,0.010157,0,63401.0,43.1,20299,-0.614926,28.8,14.1,0.5,0.7,9.2,87.4
2994,56.0,WY,Wyoming,Washakie County,56043.0,-0.007825,0,55190.0,36.0,7885,-0.640377,22.7,21.7,0.8,0.5,14.1,82.4


In [51]:
merged_data = safe_graph_data.merge(google_data, left_on = 'county_code', right_on = 'census_fips_code')

In [52]:
merged_data

,state,state_code,State Name,cnamelong,county_code,diff_in_perc_at_home,SIP?,Median Household Income,% Rural,Population_y,political_diff,% less than 18 years of age,% 65 and over,% Asian,% Black,% Hispanic,% Non-Hispanic White,census_fips_code,avg_retail_and_recreation_percent_change_6_1,avg_grocery_and_pharmacy_percent_change_6_1,avg_parks_percent_change_6_1,avg_transit_stations_percent_change_6_1,avg_workplaces_percent_change_6_1,avg_residential_percent_change_6_1
0,1.0,AL,Alabama,Autauga County,1001.0,0.050678,1,59338.0,42.0,55601,-0.494789,23.7,15.6,1.2,19.3,3.0,74.3,1001.0,-6.973684,10.026316,NaN,NaN,-25.157895,9.480000
1,1.0,AL,Alabama,Baldwin County,1003.0,0.050312,1,57588.0,42.3,218022,-0.577862,21.6,20.4,1.2,8.8,4.6,83.1,1003.0,-13.131579,4.697368,18.578947,-8.197368,-22.618421,7.605263
2,1.0,AL,Alabama,Barbour County,1005.0,0.007037,1,34382.0,67.8,24881,-0.056112,20.9,19.4,0.5,48.0,4.3,45.6,1005.0,3.755102,-7.763158,NaN,NaN,-15.631579,NaN
3,1.0,AL,Alabama,Bibb County,1007.0,0.011809,1,46064.0,68.4,22400,-0.555441,20.5,16.5,0.2,21.1,2.6,74.6,1007.0,1.971429,6.350000,NaN,NaN,-19.907895,NaN
4,1.0,AL,Alabama,Blount County,1009.0,0.038890,1,50412.0,90.0,57840,-0.813820,23.2,18.2,0.3,1.5,9.6,86.9,1009.0,-5.929825,5.105263,NaN,NaN,-23.565789,8.720000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2820,56.0,WY,Wyoming,Sweetwater County,56037.0,-0.018485,0,73315.0,10.9,43051,-0.535382,26.2,12.1,1.0,1.1,16.1,79.3,56037.0,-10.394737,7.426230,NaN,14.578947,-23.197368,7.120000
2821,56.0,WY,Wyoming,Teton County,56039.0,0.075183,1,99087.0,46.4,23081,0.278663,18.4,15.4,1.4,0.6,14.9,81.5,56039.0,-38.263158,-17.906977,-24.197368,-41.833333,-35.434211,NaN
2822,56.0,WY,Wyoming,Uinta County,56041.0,0.010157,0,63401.0,43.1,20299,-0.614926,28.8,14.1,0.5,0.7,9.2,87.4,56041.0,1.511111,15.942857,NaN,14.842105,-19.421053,NaN
2823,56.0,WY,Wyoming,Washakie County,56043.0,-0.007825,0,55190.0,36.0,7885,-0.640377,22.7,21.7,0.8,0.5,14.1,82.4,56043.0,-4.400000,-7.000000,NaN,NaN,-20.960526,NaN


We did lose about 150 counties because Google Mobility dataset only contains 2800+ counties, but I think that is okay

In [53]:
num_of_SIP_counties = merged_data['SIP?'].sum()
print(f'num of SIP counties = {num_of_SIP_counties}')

num of SIP counties = 2435


In [54]:
num_of_no_SIP_counties = merged_data[merged_data['SIP?'] == 0].shape[0]
print(f'num of no SIP counties = {num_of_no_SIP_counties}')

num of no SIP counties = 390


In [55]:
merged_data.to_csv('final_data_google_and_safe_graph_up_to_6_1_weekdays.csv', index = False)